## Importação de Bibliotecas e Módulos

Esta célula é responsável pela importação de todas as bibliotecas e módulos necessários para a execução do notebook:

- `pandas` e `numpy`: Usadas para manipulação e operações em dados numéricos e tabelas.
- `time`: Utilizada para medir o tempo de execução dos algoritmos.
- `plotly.graph_objects` e `plotly.subplots`: Ferramentas de visualização para criar gráficos interativos e subplots.
- `utils`: Módulo que contém funções auxiliares, como `build_points_df` que constrói DataFrames a partir dos resultados dos algoritmos.
- `data_pipeline`: Módulo que processa os dados necessários para os algoritmos.
- `simmulated_annealing`, `example_algorithm` e `two_opt`: Módulos que contêm os algoritmos específicos de otimização.

In [2]:
import pandas as pd
import numpy as np
import time
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from utils import build_points_df
from data_pipeline import process_data
from simmulated_annealing import simmulated_annealing
from example_algorithm import bad_algorithm
from two_opt import nearest_neighbor, two_opt_builder


## Carregamento e Preparação dos Dados

Esta célula realiza o carregamento dos dados de entrada e sua preparação inicial

In [4]:
data = pd.read_csv('../data/amostra_total.csv', sep=';')

data = process_data(data)
max_duration_hours = 6 


## Execução dos Algoritmos de Otimização

Esta célula define e executa três algoritmos de otimização para comparar seu desempenho:

- **bad_algorithm**: Executa o algoritmo padrão sem parâmetros adicionais.
- **simulated_annealing**: Utiliza o Simulated Annealing com parâmetros específicos para ajuste fino.
- **two_opt_builder**: Implementa a estratégia Two-Opt para otimização de rotas.

Cada algoritmo é executado com os seguintes passos:
1. **Medição do Tempo de Início**: Captura o tempo antes da execução do algoritmo.
2. **Execução**: O algoritmo é chamado com seus respectivos parâmetros.
3. **Medição do Tempo de Fim**: Captura o tempo após a conclusão do algoritmo.
4. **Cálculo de Métricas**: Calcula a duração média das rotas, pontos médios por rota e o tempo total de processamento.
5. **Armazenamento dos Resultados**: Os resultados são armazenados para análise posterior.

Os resultados ajudam a entender a eficiência e eficácia de cada algoritmo em termos de tempo de processamento e qualidade das rotas geradas.


In [ ]:
algos = [
    ("bad_algorithm", bad_algorithm, (data, max_duration_hours)),
    ("simulated_annealing", simmulated_annealing, (data, max_duration_hours, 1000, 2500, 0.008)), 
    ("two_opt_builder", two_opt_builder, (data, max_duration_hours))
]

results = []

for name, algo, params in algos:
    start_time = time.time()
    routes, durations = algo(*params)
    end_time = time.time()
    
    avg_duration = np.mean(durations) if durations else 0
    avg_points_per_route = np.mean([len(route) for route in routes]) if routes else 0
    processing_time = end_time - start_time
    
    results.append({
        "algorithm": name,
        "avg_duration": avg_duration,
        "avg_points_per_route": avg_points_per_route,
        "processing_time": processing_time
    })


## Visualização Comparativa dos Resultados dos Algoritmos

Esta célula cria uma figura com três subgráficos, um para cada uma das seguintes métricas:
- **Average Duration per Route**: Mostra a duração média de cada rota calculada pelos algoritmos.
- **Average Points per Route**: Exibe o número médio de pontos por rota.
- **Processing Time**: Apresenta o tempo de processamento de cada algoritmo.

### Detalhes da Visualização:
- Cada gráfico é horizontal (`orientation='h'`) para melhor comparação visual.

O objetivo desses subgráficos é fornecer uma análise visual clara e direta do desempenho dos algoritmos, facilitando a comparação entre eles.


In [19]:
avg_durations = [result['avg_duration'] for result in results]
avg_points = [result['avg_points_per_route'] for result in results]
processing_times = [result['processing_time'] for result in results]


fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=("Average Duration per Route", "Average Points per Route", "Processing Time")
)

fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=("Average Duration per Route", "Average Points per Route", "Processing Time"),
    vertical_spacing=0.15  
)

fig.add_trace(
    go.Bar(name='Avg Duration per Route', x=avg_durations, y=['Bad_Algorithm', 'Simulated_Annealing', 'Two_Opt'], orientation='h', marker=dict(line=dict(width=1))),
    row=1, col=1
)
fig.add_trace(
    go.Bar(name='Avg Points per Route', x=avg_points, y=['Bad_Algorithm', 'Simulated_Annealing', 'Two_Opt'], orientation='h', marker=dict(line=dict(width=1))),
    row=2, col=1
)
fig.add_trace(
    go.Bar(name='Processing Time', x=processing_times, y=['Bad_Algorithm', 'Simulated_Annealing', 'Two_Opt'], orientation='h', marker=dict(line=dict(width=1))),
    row=3, col=1
)

fig.update_layout(
    title_text='Comparison of Algorithm Performance',
    height=600,
    showlegend=False,
    xaxis=dict(title='', mirror=True, showline=True, linewidth=2, linecolor='black'),
    xaxis2=dict(title='', mirror=True, showline=True, linewidth=2, linecolor='black'),
    xaxis3=dict(title='', mirror=True, showline=True, linewidth=2, linecolor='black'),
    yaxis=dict(title='', mirror=True, showline=True, linewidth=2, linecolor='black'),
    yaxis2=dict(title='', mirror=True, showline=True, linewidth=2, linecolor='black'),
    yaxis3=dict(title='', mirror=True, showline=True, linewidth=2, linecolor='black')
)

fig.show()

## Análise Final e Conclusões

A análise dos resultados obtidos através dos três algoritmos de otimização de rotas revela insights importantes sobre a eficácia e eficiência de cada método:

- **Bad Algorithm**: Serviu como um ponto de referência útil, apresentando desempenhos razoáveis dado o contexto de dados limitados. Este algoritmo demonstrou ser uma base sólida para comparações futuras e otimizações.

- **Two-Opt Algorithm**: Embora tenha mostrado o melhor desempenho em termos de pontos médios por rota e duração média das rotas, seu tempo de processamento significativamente mais alto sugere a necessidade de otimização. A implementação de técnicas como Numba pode ser explorada para melhorar sua eficiência em tempo de execução, tornando-o mais viável para uso em larga escala.

- **Simulated Annealing**: Apresentou um desempenho ligeiramente inferior ao Two-Opt em termos de eficácia, mas seu tempo de processamento substancialmente menor o torna uma opção atraente para situações onde a rapidez é essencial. Esse equilíbrio entre tempo de execução e eficácia sugere que, com ajustes finos, pode ser uma alternativa competitiva, especialmente em cenários com restrições de tempo rigorosas.

### Ações Futuras

- **Aprofundamento da Análise**: Investigar mais a fundo as características dos dados que influenciam o desempenho de cada algoritmo, possibilitando ajustes mais precisos nos parâmetros de cada método.
- **Otimização do Two-Opt**: Explorar a aplicação de Numba para otimização de código, buscando reduzir o tempo de processamento sem comprometer a qualidade dos resultados.
- **Experimentação com Parâmetros**: Para o Simulated Annealing, experimentar com diferentes configurações de parâmetros pode revelar melhorias potenciais que equilibrem ainda mais tempo de execução e eficácia.

A visualização dos resultados, como mostrado nos gráficos, facilita a compreensão do desempenho relativo de cada algoritmo.


# Análise dos Algoritmos com seleção de dados aleatória e Parâmetros diferentes

In [22]:
data = pd.read_csv('../data/amostra_total.csv', sep=';')

data = process_data(data, 0.0015)

In [ ]:
algos = [
    ("bad_algorithm", bad_algorithm, (data, max_duration_hours)),
    ("simulated_annealing", simmulated_annealing, (data, max_duration_hours, 2000, 3000, 0.015)), 
    ("two_opt_builder", two_opt_builder, (data, max_duration_hours))
]

results = []

for name, algo, params in algos:
    start_time = time.time()
    routes, durations = algo(*params)
    end_time = time.time()
    
    avg_duration = np.mean(durations) if durations else 0
    avg_points_per_route = np.mean([len(route) for route in routes]) if routes else 0
    processing_time = end_time - start_time
    
    results.append({
        "algorithm": name,
        "avg_duration": avg_duration,
        "avg_points_per_route": avg_points_per_route,
        "processing_time": processing_time
    })

In [24]:
avg_durations = [result['avg_duration'] for result in results]
avg_points = [result['avg_points_per_route'] for result in results]
processing_times = [result['processing_time'] for result in results]


fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=("Average Duration per Route", "Average Points per Route", "Processing Time")
)

fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=("Average Duration per Route", "Average Points per Route", "Processing Time"),
    vertical_spacing=0.15  
)

fig.add_trace(
    go.Bar(name='Avg Duration per Route', x=avg_durations, y=['Bad_Algorithm', 'Simulated_Annealing', 'Two_Opt'], orientation='h', marker=dict(line=dict(width=1))),
    row=1, col=1
)
fig.add_trace(
    go.Bar(name='Avg Points per Route', x=avg_points, y=['Bad_Algorithm', 'Simulated_Annealing', 'Two_Opt'], orientation='h', marker=dict(line=dict(width=1))),
    row=2, col=1
)
fig.add_trace(
    go.Bar(name='Processing Time', x=processing_times, y=['Bad_Algorithm', 'Simulated_Annealing', 'Two_Opt'], orientation='h', marker=dict(line=dict(width=1))),
    row=3, col=1
)

fig.update_layout(
    title_text='Comparison of Algorithm Performance',
    height=600,
    showlegend=False,
    xaxis=dict(title='', mirror=True, showline=True, linewidth=2, linecolor='black'),
    xaxis2=dict(title='', mirror=True, showline=True, linewidth=2, linecolor='black'),
    xaxis3=dict(title='', mirror=True, showline=True, linewidth=2, linecolor='black'),
    yaxis=dict(title='', mirror=True, showline=True, linewidth=2, linecolor='black'),
    yaxis2=dict(title='', mirror=True, showline=True, linewidth=2, linecolor='black'),
    yaxis3=dict(title='', mirror=True, showline=True, linewidth=2, linecolor='black')
)

fig.show()

## Resultados Obtidos com Outra Combinação de Dados e Parâmetros

Após a análise inicial, realizamos uma segunda rodada de testes, expandindo o conjunto de dados em 50% e selecionando outras rotas aleatoriamente dos dados iniciais. Além disso, os parâmetros do Simulated Annealing foram ajustados para explorar diferentes configurações de temperatura e taxas de resfriamento.

### Observações:

- **Consistência dos Resultados**: Os resultados desta iteração foram notavelmente semelhantes aos obtidos anteriormente, reforçando as conclusões iniciais sobre o desempenho relativo dos algoritmos.

- **Two-Opt**: Continuou a apresentar o melhor desempenho em termos de eficiência da rota, mas seu tempo de processamento ainda destaca a necessidade de otimização para torná-lo mais aplicável em um contexto operacional real.

- **Simulated Annealing**: As modificações nos parâmetros não resultaram em melhorias significativas na eficácia ou eficiência do algoritmo, sugerindo que os ajustes nas configurações iniciais já estavam próximos do ótimo para o conjunto de dados considerado.

- **Bad Algorithm**: Apesar do aumento no conjunto de dados e da seleção aleatória de rotas, este algoritmo básico manteve um desempenho consistente, servindo bem como uma linha de base para comparação.

### Análise:

Esta consistência nos resultados sugere que os algoritmos são consistentes em relação à variação dos dados, com pequenas alterações nos parâmetros tendo impacto limitado na performance geral. Isso indica a necessidade de investigações mais profundas e talvez mais especializadas em termos de otimização de parâmetros ou até mesmo a adoção de estratégias de aprendizado de máquina para a seleção automática dos melhores parâmetros e configurações de algoritmos, que serão feitas na próxima *sprint*.

